# **Tensorflow**
# key : tf.modulename.functionanae(parameters)
# key :  tensor <=>anydata  (yasari any data lai tensor(matrix) ra any tensor(matrix) lai data ma convert garxa  
[tensofflow for python ](https://www.tensorflow.org/)

# **STEPS** (TensorFlow ecosystem) [read step by step form tensorflow ecosystem](https://www.tensorflow.org/learn)
#C=create input piplines/Load & preprocess data
#R=Read Data
#U=update parameter /Build, train & reuse models
#D=Deploy model

# **some important note on doing data preprocessing**
 # 1.simply print the data eg:: **data**
 # 2.print data with print function eg:: **print(data)**
 # 3.print the shape of the data eg::**print(data.shape)**
 # 4.check the data sequence eg:: **list,tuple,set,array,dictionary**
 # 5.check the data type of elements eg::**int,float,string**

# STEP 1:: C=Create input pipline/load & preprocess data

# A. **[tf.data](https://www.tensorflow.org/api_docs/python/tf/data)**  api enables you to build complex input pipline from single reusable pices
# B.**[tf.data.Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset)**  abstraction represents a sequence of elements ,in which each element consists of one or more components .eg image pipline
 # i. There is two distinct ways to create Dataset .
 # * A data source construct a Dataset from data stored in memory or in one or more files.
 # * A data transformation constructs a datasets a dataset from one or more tf.data.Dataset

In [0]:
import tensorflow as tf
from __future__ import  absolute_import,division,print_function,unicode_literals

In [12]:
try :
  !pip install -q tf-nightly
except Exception:
    pass 


     |████████████████████████████████| 511.7MB 18kB/s 
     |████████████████████████████████| 3.9MB 35.4MB/s 
     |████████████████████████████████| 460kB 52.0MB/s 
     |████████████████████████████████| 2.9MB 36.0MB/s 
ERROR: tensorflow 1.15.0 has requirement gast==0.2.2, but you'll have gast 0.3.3 which is incompatible.


In [13]:
print(tf.__version__)

1.15.0


In [0]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
np.set_printoptions(precision=4)

# To create an input pipeline ,you must start with source .
# * to construct a Dataset from data in memory ,you can use 
# **tf.data.Dataset.from_tensors()** or **tf.data.Dataset.from_tensor_slices()**

In [15]:
dataset =tf.data.Dataset.from_tensor_slices([1,2,3,4])
dataset

<DatasetV1Adapter shapes: (), types: tf.int32>

In [16]:
for i in dataset:
  print(i.numpy())

RuntimeError: ignored

In [0]:
it=iter(dataset) # iterator le tensor as a  input linxa and auta auta gardai element denxa.
print(next(it).numpy())

In [0]:
print(dataset.reduce(0,lambda state,value:state + value).numpy()) #reduce the all elements to produce a single result.

# Dataset Structure
# * **tf.TypeSpace** ::kun type ko ho tensor Tensor,SparseTensor,RaggedTensor,TensorArray or Dataset   Dinxa
# * **Dataset.element_spec** ::tensor ko dataset ko type of each element component denaxa 
# **note :** shape(n,)  n vanyasi dimension janauxa  x,y =2 dimension..
 # so yaha n le no of column  which represent the  dimension.

In [0]:
tf.TypeSpec


In [0]:
dataset=tf.data.Dataset.from_tensor_slices(tf.random.uniform([4,10]))
# for i in dataset:
#    print(i.numpy())
dataset.element_spec # NOTE::
#print(dataset)

In [0]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random.uniform([4]),
    tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))

dataset2.element_spec


In [0]:
dataset3=tf.data.Dataset.zip((dataset,dataset2))
dataset3.element_spec

In [0]:
# dataset containing a sparse tensor 
dataset4=tf.data.Dataset.from_tensors(tf.SparseTensor(indices=[[0,0],[1,2]],values=[1,2],dense_shape=[3,4]))
dataset4.element_spec

In [0]:
# use value_type to see the type of value represented by element spec
dataset4.element_spec.value_type

In [0]:
dataset5=tf.data.Dataset.from_tensor_slices(tf.random.uniform([4,10],maxval=10,dtype=tf.int32))
dataset5

In [0]:
for z in dataset5:
  print(z.numpy())

In [0]:
dataset6 = tf.data.Dataset.from_tensor_slices(
   (tf.random.uniform([4]),
    tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))

dataset6

# Reading input data 
# * consuming numpy arrays

In [0]:
# LOAD FROM .npz file 
DATA_URL='https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz'
path=tf.keras.utils.get_file('mnist.npz',DATA_URL)
with np.load(path) as data:
  train_examples=data['x_train']
  train_labels=data['y_train']
  test_examples=data['x_test']
  test_labels=data['y_test']

![iamge here](https://raw.githubusercontent.com/themadan/1.DataScience-libraries/master/Tensorflow(nn-deep%20learning)/1.tensorflow%20ecosystem/images/ndarray%20and%20shape.png)

In [0]:
train,test=tf.keras.datasets.fashion_mnist.load_data() # mnist data load garxa ani tuple ko rupma return garxa (x_train,y_train),(x_test,y_test) yasari return garxa.
print(train[0].shape) #x_train   (60000=no of row=no of image,28=no of row ,28=no of column)
print(train[1].shape)# y_train(labels)
print(test[0].shape)# x_test
print(test[1].shape)#y_test
print(train[0][0].shape)


In [0]:
images,labels=train
images=images/255


In [0]:
dataset=tf.data.Dataset.from_tensor_slices((images,labels))
dataset 

# Consuming Python Generators
# * another common data source that can easily be ingested as a tf.data.Dataset is a python generator

In [0]:
def count(stop):
  i=0
  while i<stop:
    yield i
    i+=1

In [0]:
for n in count(5):
  print(n)

In [0]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [0]:
img_gen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,rotation_range=20)


In [0]:
images,labels=next(img_gen.flow_from_directory(flowers))

In [0]:
print(images.dtype,images.shape)
print(labels.dtype,labels.shape)

# Consuming TFRecord data

In [0]:
# Creates a dataset that reads all of the examples from two files.
fsns_test_file = tf.keras.utils.get_file("fsns.tfrec", "https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001")

In [0]:
dataset=tf.data.TFRecordDataset(filenames=[fsns_test_file])
dataset

In [17]:
raw_example=next(iter(dataset))
parsed=tf.train.Example.FromString(raw_example.numpy())
parsed.features.feature['iamge/text']

RuntimeError: ignored